In [ ]:
from sliding_window_validation import SlidingWindowValidator
from gp_utils import Period
training_period = Period('2018/04/01 00:00:00 UTC', '2018/05/01 00:00:00 UTC')
validation_period = Period('2018/05/01 00:00:00 UTC', '2018/06/01 00:00:00 UTC')
end_time = '2018/10/21 00:00:00 UTC'
step = 60*60*24*7

val = SlidingWindowValidator('gv5_experiments_sliding_template.json')
df = val.run(training_period, validation_period, step, end_time, 'sliding_window_experiments_update_5')

In [ ]:
df

In [ ]:
from gp_artemis import ExperimentManager
from chart_plotter import get_dot_graph, save_dot_graph
import pandas as pd
from PIL import Image

pd.set_option('display.max_colwidth', -1)


def get_thumbnail(path):
    i = Image.open(path)
    i.thumbnail((400, 400), Image.LANCZOS)
    return i

def image_base64(im):
    if isinstance(im, str):
        im = get_thumbnail(im)
    with BytesIO() as buffer:
        im.save(buffer, 'png')
        return base64.b64encode(buffer.getvalue()).decode()

def image_formatter(im):
    return f'<img src="data:image/jpeg;base64,{image_base64(im)}">'



images = []
for i in range(df.shape[0]):
    row = df.iloc[i]
    training_start, training_end = row.training_period.split(' - ')
    validation_start, validation_end = row.validation_period.split(' - ')


    experiment_json = val.experiment_json_template.format(start_time=training_start, end_time=training_end)
    e = ExperimentManager(experiment_container=experiment_json, read_from_file=False)
    experiment_id = 'run_evolution.d_BTC-USDT-1522540800-1525132800;gv5;ff_benchmarkdiff;x_0.9;m_0.7;n_500;gen_10;td_5;a;nrs'

    gp = e.build_genetic_program(data=None, function_provider=e.function_provider, db_record=e.get_db_record_from_experiment_id(experiment_id))
    individual = gp.individual_from_string(row.strategy[len('Strategy: evolved using genetic programming\nRule set: '):])
    save_dot_graph(individual, 'tmp', format='png')
    img = get_thumbnail('tmp.png')
    images.append(img)

df[image] = images

In [ ]:
from IPython.display import HTML
from io import BytesIO
import base64
HTML(df[['training_all_coins_trading_against_BTC', 'image']].to_html(formatters={'image': image_formatter}, escape=False))

In [ ]:
writer = pd.ExcelWriter('tmp.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()